# FINAL PROJECT

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# Load clean data
fomc_rates = pd.read_csv('inputs/Clean_Data/Clean_Rates.csv')
sp500_ret = pd.read_csv('inputs/Clean_Data/Clean_sp500_ret.csv')
stock_tv = pd.read_csv('inputs/Clean_Data/Clean_stock_tv.csv')
Vbilx = pd.read_csv('inputs/Clean_Data/Clean_Vbilx.csv')
Vbirx= pd.read_csv('inputs/Clean_Data/Clean_Vbirx.csv')

In [3]:
sp500_ret = sp500_ret.rename(columns = {'date': 'Date','Daily Returns' : 'SP_daily_ret', 'Price': 'SP_daily_price' })

In [4]:
Vbirx
Vbirx = Vbirx.rename(columns = {'Daily Returns' : 'VBIRX_daily_ret', 'Price': 'VBIRX_daily_price' })
Vbirx = Vbirx.rename(columns = {'Daily Price': 'VBIRX_daily_price' })
Vbirx

,VBIRX_daily_ret,VBIRX_daily_price,Date
0,0.000000,9.148485,2015-01-02
1,0.000954,9.157212,2015-01-05
2,0.000952,9.165927,2015-01-06
3,0.000951,9.174645,2015-01-07
4,-0.000950,9.165927,2015-01-08
...,...,...,...
2086,0.000000,9.990000,2023-04-18
2087,-0.001001,9.980000,2023-04-19
2088,0.002004,10.000000,2023-04-20
2089,0.000000,10.000000,2023-04-21


In [5]:
Vbilx = Vbilx.rename(columns = {'Daily Price': 'VBILX_daily_price', 'Daily Returns' : 'VBLIX_daily_ret' })
Vbilx

,VBLIX_daily_ret,VBILX_daily_price,Date
0,0.004348,9.135239,2015-01-05
1,0.004330,9.174790,2015-01-06
2,0.000000,9.174790,2015-01-07
3,-0.002586,9.151061,2015-01-08
4,0.003457,9.182695,2015-01-09
...,...,...,...
2085,0.001938,10.340000,2023-04-18
2086,-0.001934,10.320000,2023-04-19
2087,0.003876,10.360000,2023-04-20
2088,-0.001930,10.340000,2023-04-21


In [6]:
sp500_ret = sp500_ret.rename(columns = {'returns' : 'sp500_ret'})
sp500_ret

,Date,S&P_500,sp500_ret
0,2015-01-05,2020.58,-0.018278
1,2015-01-06,2002.61,-0.008893
2,2015-01-07,2025.90,0.011630
3,2015-01-08,2062.14,0.017888
4,2015-01-09,2044.81,-0.008404
...,...,...,...
2085,2023-04-18,4154.87,0.000855
2086,2023-04-19,4154.52,-0.000084
2087,2023-04-20,4129.79,-0.005953
2088,2023-04-21,4133.52,0.000903


In [7]:
stock_tv
stock_tv.drop_duplicates(inplace=True)
stock_tv = stock_tv.rename(columns = {'Value' : 'sp500_tv'})
stock_tv

,Date,sp500_tv
0,12/2/15,1088000.0
1,12/3/15,1379000.0
2,12/4/15,1308000.0
3,12/7/15,712744.0
4,12/8/15,670998.0
...,...,...
502,3/31/23,3293000.0
503,4/3/23,2494000.0
504,4/4/23,2687000.0
505,4/5/23,2311000.0


## Merging data 

In [8]:
final_data = sp500_ret.merge(Vbilx, 
              on=['Date'],
              how='left',
              validate='one_to_one') # or 'many_to_one'

In [9]:
final_data2 = final_data.merge(Vbirx, on=['Date'],
              how='left',
              validate='one_to_one')

In [13]:
all = final_data2.merge(fomc_rates, on = ["Date"], how = 'left')
all

,Date,S&P_500,sp500_ret,VBLIX_daily_ret,VBILX_daily_price,VBIRX_daily_ret,VBIRX_daily_price,Increase,Decrease,Tgt_level
0,2015-01-05,2020.58,-0.018278,0.004348,9.135239,0.000954,9.157212,NaN,NaN,NaN
1,2015-01-06,2002.61,-0.008893,0.004330,9.174790,0.000952,9.165927,NaN,NaN,NaN
2,2015-01-07,2025.90,0.011630,0.000000,9.174790,0.000951,9.174645,NaN,NaN,NaN
3,2015-01-08,2062.14,0.017888,-0.002586,9.151061,-0.000950,9.165927,NaN,NaN,NaN
4,2015-01-09,2044.81,-0.008404,0.003457,9.182695,0.001903,9.183371,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2085,2023-04-18,4154.87,0.000855,0.001938,10.340000,0.000000,9.990000,NaN,NaN,NaN
2086,2023-04-19,4154.52,-0.000084,-0.001934,10.320000,-0.001001,9.980000,NaN,NaN,NaN
2087,2023-04-20,4129.79,-0.005953,0.003876,10.360000,0.002004,10.000000,NaN,NaN,NaN
2088,2023-04-21,4133.52,0.000903,-0.001930,10.340000,0.000000,10.000000,NaN,NaN,NaN


**Adding Event Study Variable**

In [11]:
Vbilx =Vbilx.reset_index()

In [12]:
for index, row in tqdm(rates.iterrows()):
    try:
        filing_index = Vbilx.loc[(Vbilx['Date'] == row['Date'])].index[0]
        forward_returns = Vbilx.loc[filing_index:filing_index+15, 'Daily Returns']
        negative_returns = Vbilx.loc[filing_index-15:filing_index, 'Daily Returns']
        forward_returns = forward_returns +1
        negative_returns = negative_returns +1
        forward_returns = forward_returns.cumprod()
        negative_returns = negative_returns.cumprod()
        forward_returns = forward_returns.to_list()
        negative_returns = negative_returns.to_list()
        forward_cumret = forward_returns[2]
        negative_cumret = negative_returns[2]
        forward_cumret = forward_cumret -1
        negative_cumret = negative_cumret -1
        rates.loc[index, 'Vbilx: T-T+15'] = forward_cumret
        rates.loc[index, 'Vbilx: T-15-T'] = negative_cumret
    except IndexError:
        pass

NameError: name 'rates' is not defined

In [ ]:
for index, row in tqdm(rates.iterrows()):
    try:
        filing_index = Vbirx.loc[(Vbirx['Date'] == row['Date'])].index[0]
        forward_returns = Vbirx.loc[filing_index:filing_index+15, 'Daily Returns']
        negative_returns = Vbirx.loc[filing_index-15:filing_index, 'Daily Returns']
        forward_returns = forward_returns +1
        negative_returns = negative_returns +1
        forward_returns = forward_returns.cumprod()
        negative_returns = negative_returns.cumprod()
        forward_returns = forward_returns.to_list()
        negative_returns = negative_returns.to_list()
        forward_cumret = forward_returns[2]
        negative_cumret = negative_returns[2]
        forward_cumret = forward_cumret -1
        negative_cumret = negative_cumret -1
        rates.loc[index, 'Vbirx: T-T+15'] = forward_cumret
        rates.loc[index, 'Vbirx: T-15-T'] = negative_cumret
    except IndexError:
        pass

In [ ]:
for index, row in tqdm(rates.iterrows()):
    try:
        filing_index = stock_tv.loc[(stock_tv['Date'] == row['Date'])].index[0]
        forward_returns = stock_tv.loc[filing_index:filing_index+15, 'Daily Returns']
        negative_returns = stock_tv.loc[filing_index-15:filing_index, 'Daily Returns']
        forward_returns = forward_returns +1
        negative_returns = negative_returns +1
        forward_returns = forward_returns.cumprod()
        negative_returns = negative_returns.cumprod()
        forward_returns = forward_returns.to_list()
        negative_returns = negative_returns.to_list()
        forward_cumret = forward_returns[2]
        negative_cumret = negative_returns[2]
        forward_cumret = forward_cumret -1
        negative_cumret = negative_cumret -1
        rates.loc[index, 'SP500 Volume: T-T+15'] = forward_cumret
        rates.loc[index, 'SP500 Volume: T-15-T'] = negative_cumret
    except IndexError:
        pass

In [ ]:
for index, row in tqdm(rates.iterrows()):
    try:
        filing_index = sp500_ret.loc[(sp500_ret['Date'] == row['Date'])].index[0]
        forward_returns = sp500_ret.loc[filing_index:filing_index+15, 'returns']
        negative_returns = sp500_ret.loc[filing_index-15:filing_index, 'returns']
        forward_returns = forward_returns +1
        negative_returns = negative_returns +1
        forward_returns = forward_returns.cumprod()
        negative_returns = negative_returns.cumprod()
        forward_returns = forward_returns.to_list()
        negative_returns = negative_returns.to_list()
        forward_cumret = forward_returns[2]
        negative_cumret = negative_returns[2]
        forward_cumret = forward_cumret -1
        negative_cumret = negative_cumret -1
        rates.loc[index, 'SP500: T-T+15'] = forward_cumret
        rates.loc[index, 'SP500: T-15-T'] = negative_cumret
    except IndexError:
        pass

In [ ]:
rates

In [ ]:
rates.to_csv('inputs/Final_Data/Final_Dataset.csv', index=True)

In [ ]:
# Set date column as index in each dataset
Vbilx.set_index('Date', inplace=True)
Vbirx.set_index('Date', inplace=True)
sp500_ret.set_index('Date', inplace=True)

# Merge datasets 
merged_data = pd.concat([Vbilx['Daily Price'],Vbilx['Daily Returns'], Vbirx['Daily Price'], Vbirx['Daily Returns'], sp500_ret['S&P_500']], axis=1, keys=['VBILX Price','VBILX Returns', 'VBIRX Price', 'VBIRX Returns', 'S&P 500'])
merged_data.reset_index(inplace=True)
merged_data = merged_data.rename(columns={'index':'Date'})

In [ ]:
merged_data.head()

## Merge for FOMC Meeting days

In [ ]:
filtered_dates = rates['Date'].tolist()
merged_data_filtered = merged_data[merged_data['Date'].isin(filtered_dates)]

In [ ]:
Data_with_rates = pd.merge(merged_data_filtered, rates, on='Date', how='left')

In [ ]:
Data_with_rates  = Data_with_rates .rename(columns={'Increase': 'rates increase', 'Decrease': 'rates decrease', 'Tgt_level': 'rates Tgt_level'})
Data_with_rates.head()

## Correlation matrix 

In [ ]:
corr_matrix = Data_with_rates.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Data with rates')
plt.show()

## T+15 / T-15 FOMC Meetings

In [ ]:
#change to datetime
merged_data['Date'] = pd.to_datetime(merged_data['Date'])
rates['Date'] = pd.to_datetime(rates['Date'])

fifteen_days = pd.DataFrame()
filtered_dates = rates['Date'].tolist()


# Loop 
for date in filtered_dates:
    # Convert date to datetime object
    match_date = pd.to_datetime(date)
    # Filter the merged data to the 15 days prior and after the matching date
    prior_15 = merged_data.loc[(merged_data['Date'] >= match_date - pd.Timedelta(days=15)) & (merged_data['Date'] <= match_date)]
    post_15 = merged_data.loc[(merged_data['Date'] > match_date) & (merged_data['Date'] <= match_date + pd.Timedelta(days=15))]
    # Combine the filtered dataframes and append to the results dataframe
    filtered_data = pd.concat([prior_15, post_15])
    fifteen_days = pd.concat([fifteen_days, filtered_data])

# Sort results and drop and duplicates
fifteen_days.sort_values(by='Date', inplace=True)
fifteen_days.drop_duplicates(inplace=True)
fifteen_days.head()

#### NOTE: the date is not exact because there is no data for some days. for example 2015-12-12 and 2015-12-13 are not in the data. I just think it's important to note that! 

In [ ]:
Data_with_rates.columns

## Event Study Graphs

In [ ]:
sns.scatterplot(x="Date", y="S&P 500", data=Data_with_rates)#hue='group')

### Data Sources

- FOMC Rates 
        Source:https://www.federalreserve.gov/monetarypolicy/openmarket.htm
        
- Index Returns
    Source: https://www.spglobal.com/spdji/en/indices/equity/sp-500/#overview
    
- Stock Trading Volume¶
  Source: https://ycharts.com/indicators/cboe_spx_volume
  
- VBLAX data 
    Source: Yahoo Finance (import yfinance as yf)
    
- VBIRX data 
    Source: Yahoo Finance (import yfinance as yf)